<a href="https://colab.research.google.com/github/Reshoan/Excel-Summary/blob/main/Excel_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
%%writefile app.py
from pyngrok import ngrok
import streamlit as st
import pandas as pd
from io import BytesIO
import re
import time
from openpyxl import load_workbook
import base64
from collections import deque, defaultdict

# Try to import graphviz python package
try:
    import graphviz
    _GRAPHVIZ_AVAILABLE = True
except Exception:
    _GRAPHVIZ_AVAILABLE = False

# ------ STREAMLIT APP ------ #
st.title("Excel Form Processor")

# Step 1: Upload Excel file
uploaded_file = st.file_uploader("Upload an Excel file", type=["xlsx"])

if uploaded_file:
    xls = pd.ExcelFile(uploaded_file)

    # Must contain formFields
    if "formFields" not in xls.sheet_names:
        st.error("The Excel file must contain a sheet named 'formFields'.")
    else:

        df = pd.read_excel(uploaded_file, sheet_name="formFields")

        # Fixed required columns
        field_name_col = "name"
        field_type_col = "input_type"
        field_order_col = "order"
        form_col = "formName"

        # Fixed category columns
        fixed_category_cols = [
            "is_editable", "is_hidden", "is_required", "is_nullable",
            "is_multiselect", "is_richtext", "editor_height",
            "is_encrypted", "is_conditional"
        ]
        selected_category_cols = [c for c in fixed_category_cols if c in df.columns]

        # Load fieldMapping sheet
        field_mapping_df = pd.DataFrame(columns=["formName", "fieldName"])
        if "fieldMapping" in xls.sheet_names:
            field_mapping_df = pd.read_excel(uploaded_file, sheet_name="fieldMapping")

        # Process all unique forms
        all_forms = sorted(df[form_col].dropna().unique())

        # --- TOTAL sheet count = form sheets + Transition_mapping + Transition_mapping_visio ---
        total_sheet_count = len(all_forms) + 2

        st.write(f"Processing {total_sheet_count} sheets automatically...")

        # Live-updating numbered sheet list
        sheet_list_placeholder = st.empty()

        output = BytesIO()
        progress_bar = st.progress(0)

        uploaded_file.seek(0)
        wb = load_workbook(uploaded_file)

        # Remove existing mapping/diagram sheets if exist
        for sheet_to_remove in ["Transition_mapping", "Transition_mapping_visio"]:
            if sheet_to_remove in wb.sheetnames:
                wb.remove(wb[sheet_to_remove])

        created_sheet_names = []  # list to show sheets live

        # --------------------------- #
        # Generate form-based sheets
        # --------------------------- #
        for i, full_form_name in enumerate(all_forms):

            parts = full_form_name.split("_")
            if len(parts) > 3:
                middle_parts = parts[2:-1]
                short_name = "_".join(middle_parts)
            else:
                short_name = full_form_name

            safe_short_name = re.sub(r'[\\/*?:\[\]]', '_', short_name)
            safe_short_name = safe_short_name[:31]

            form_df = df[df[form_col] == full_form_name].sort_values(by=field_order_col)

            final_df = form_df[[field_name_col, field_type_col, field_order_col]].copy()

            # Repeater column
            repeater_list = []
            filtered_mapping = field_mapping_df[field_mapping_df['formName'] == full_form_name]
            mapped_fields = set(filtered_mapping['fieldName'].dropna())

            for field in final_df[field_name_col]:
                repeater_list.append("Yes" if field in mapped_fields else "")

            final_df.insert(len(final_df.columns), "Repeater", repeater_list)

            # Categories column
            if selected_category_cols:
                categories_list = []
                for idx, row in form_df.iterrows():
                    selected = [col for col in selected_category_cols if row.get(col) == 1]
                    selected.sort()
                    categories_list.append(",".join(selected) if selected else "")
                final_df["Categories"] = categories_list

            # Remove existing sheet
            if safe_short_name in wb.sheetnames:
                wb.remove(wb[safe_short_name])

            # Create new sheet
            ws = wb.create_sheet(title=safe_short_name)

            # Write header A1 & B1
            ws.cell(row=1, column=1, value="Form")
            ws.cell(row=1, column=2, value=full_form_name)

            # Column headers row 2
            for c_idx, header in enumerate(final_df.columns, start=1):
                ws.cell(row=2, column=c_idx, value=header)

            # Data rows from row 3
            for r_idx, row in enumerate(final_df.values, start=3):
                for c_idx, value in enumerate(row, start=1):
                    ws.cell(row=r_idx, column=c_idx, value=value)

            # Track created sheet
            created_sheet_names.append(safe_short_name)

            # Live updating numbered sheet list
            numbered_list = [f"{idx+1}. {name}" for idx, name in enumerate(created_sheet_names)]
            sheet_list_placeholder.markdown(
                "<div style='max-height:200px; overflow-y:auto; border:1px solid #444; padding:10px; border-radius:5px;'>"
                + "<br>".join(numbered_list)
                + "</div>",
                unsafe_allow_html=True
            )

            # Progress update
            progress_bar.progress(int((len(created_sheet_names) / total_sheet_count) * 100))
            time.sleep(0.05)

        # ----------------------------------------------------------- #
        #                   TRANSITION MAPPING PART
        # ----------------------------------------------------------- #

        diagram_generated = False
        diagram_bytes = None

        if "transitions" not in xls.sheet_names:
            st.error("No 'transitions' sheet found. Cannot build Transition_mapping.")
        else:
            transitions_df = pd.read_excel(uploaded_file, sheet_name="transitions")

            # Keep only needed columns if present
            needed_cols = ["name", "workflowFromStateName", "workflowToStateName", "workflowFormName", "fromToStateName"]
            present_cols = [c for c in needed_cols if c in transitions_df.columns]
            transitions_df = transitions_df[present_cols].copy()

            # Reorder to expected if available
            reorder = [c for c in needed_cols if c in transitions_df.columns]
            transitions_df = transitions_df[reorder]

            # case-insensitive sort
            if "workflowFromStateName" in transitions_df.columns and "workflowToStateName" in transitions_df.columns:
                transitions_df = transitions_df.sort_values(
                    by=["workflowFromStateName", "workflowToStateName"],
                    key=lambda col: col.str.lower()
                )

            # Add SLA columns initially empty if not present
            if "sla_time" not in transitions_df.columns:
                transitions_df["sla_time"] = ""
            if "sla_time_type" not in transitions_df.columns:
                transitions_df["sla_time_type"] = ""

            if "workflowSlas" in xls.sheet_names:
                slas_df = pd.read_excel(uploaded_file, sheet_name="workflowSlas")
                # If slas_df has expected columns, merge values
                if {"workflowFromStateName", "workflowToStateName", "sla_time", "sla_time_type"}.issubset(slas_df.columns):
                    for idx, row in transitions_df.iterrows():
                        from_state = row["workflowFromStateName"]
                        to_state = row["workflowToStateName"]

                        match = slas_df[
                            (slas_df["workflowFromStateName"] == from_state) &
                            (slas_df["workflowToStateName"] == to_state)
                        ]

                        if not match.empty:
                            transitions_df.at[idx, "sla_time"] = match.iloc[0]["sla_time"]
                            transitions_df.at[idx, "sla_time_type"] = match.iloc[0]["sla_time_type"]

            # Final column order if possible
            final_order = ["name", "workflowFromStateName", "workflowToStateName", "workflowFormName", "sla_time", "sla_time_type", "fromToStateName"]
            final_cols = [c for c in final_order if c in transitions_df.columns]
            transitions_df = transitions_df[final_cols]

            ws = wb.create_sheet("Transition_mapping")

            for c_idx, header in enumerate(transitions_df.columns, start=1):
                ws.cell(row=1, column=c_idx, value=header)

            for r_idx, row in enumerate(transitions_df.values, start=2):
                for c_idx, value in enumerate(row, start=1):
                    ws.cell(row=r_idx, column=c_idx, value=value)

            # Add Transition sheet to live list
            created_sheet_names.append("Transition_mapping")
            numbered_list = [f"{idx+1}. {name}" for idx, name in enumerate(created_sheet_names)]
            sheet_list_placeholder.markdown(
                "<div style='max-height:200px; overflow-y:auto; border:1px solid #444; padding:10px; border-radius:5px;'>"
                + "<br>".join(numbered_list)
                + "</div>",
                unsafe_allow_html=True
            )

            # Final progress update (not final yet: diagram follows)
            progress_bar.progress(int((len(created_sheet_names) / total_sheet_count) * 100))

            # ---------------------------
            # Build Transition_mapping_visio (table) and Flowchart (PDF)
            # ---------------------------
            # Build directed graph from transitions_df
            edges = defaultdict(list)   # edges[from] = list of (to, transition_name)
            nodes = set()

            if "workflowFromStateName" in transitions_df.columns and "workflowToStateName" in transitions_df.columns:
                for _, row in transitions_df.iterrows():
                    frm = str(row["workflowFromStateName"]).strip()
                    to = str(row["workflowToStateName"]).strip()
                    name = str(row["name"]).strip() if "name" in transitions_df.columns else ""
                    nodes.add(frm)
                    nodes.add(to)
                    edges[frm].append((to, name))

            # Determine start node
            start_node = None
            if "Product Requisition" in nodes:
                start_node = "Product Requisition"
            else:
                incoming = set()
                for frm, lst in edges.items():
                    for to, _ in lst:
                        incoming.add(to)
                candidates = [n for n in nodes if n not in incoming]
                start_node = candidates[0] if candidates else (sorted(nodes)[0] if nodes else None)

            # BFS ordering to create stable node list
            node_order = []
            seen = set()
            if start_node:
                q = deque([start_node])
                seen.add(start_node)
                while q:
                    n = q.popleft()
                    node_order.append(n)
                    for to, _ in edges.get(n, []):
                        if to not in seen:
                            seen.add(to)
                            q.append(to)
                for n in sorted(nodes):
                    if n not in seen:
                        node_order.append(n)
            else:
                node_order = sorted(nodes)

            # Assign Process Step IDs: P100, P200, ...
            id_map = {}
            base = 100
            for idx, node in enumerate(node_order):
                id_map[node] = f"P{(idx+1)*base}"

            # Build diagram rows for the Visio-style sheet
            diagram_rows = []
            for node in node_order:
                desc = node
                targets = edges.get(node, [])
                if targets:
                    tgt_map = defaultdict(list)
                    for to, tname in targets:
                        tgt_map[to].append(tname if tname else "")
                    ordered_targets = sorted(tgt_map.keys(), key=lambda s: s.lower())
                    next_ids = [id_map.get(t, "") for t in ordered_targets]
                    connector_labels = [", ".join([n for n in tgt_map[t] if n]) for t in ordered_targets]
                    next_ids_str = ",".join(next_ids)
                    connector_str = ",".join(connector_labels)
                else:
                    next_ids_str = ""
                    connector_str = ""

                low = node.lower() if isinstance(node, str) else ""
                if node == "Product Requisition":
                    shape = "Start"
                elif ("reject" in low) or ("closed" in low):
                    shape = "End"
                else:
                    shape = "Process"

                diagram_rows.append({
                    "Process Step ID": id_map.get(node, ""),
                    "Process Step Description": desc,
                    "Next Step ID": next_ids_str,
                    "Connector Label": connector_str,
                    "Shape Type": shape,
                    "Alt Text": ""   # new empty column requested
                })

            # Write Transition_mapping_visio sheet
            visio_sheet_name = "Transition_mapping_visio"
            if visio_sheet_name in wb.sheetnames:
                wb.remove(wb[visio_sheet_name])
            wsd = wb.create_sheet(visio_sheet_name)
            headers = ["Process Step ID", "Process Step Description", "Next Step ID", "Connector Label", "Shape Type", "Alt Text"]
            for c_idx, header in enumerate(headers, start=1):
                wsd.cell(row=1, column=c_idx, value=header)

            for r_idx, row in enumerate(diagram_rows, start=2):
                for c_idx, header in enumerate(headers, start=1):
                    wsd.cell(row=r_idx, column=c_idx, value=row.get(header, ""))

            # Add visio sheet to live list
            created_sheet_names.append(visio_sheet_name)
            numbered_list = [f"{idx+1}. {name}" for idx, name in enumerate(created_sheet_names)]
            sheet_list_placeholder.markdown(
                "<div style='max-height:200px; overflow-y:auto; border:1px solid #444; padding:10px; border-radius:5px;'>"
                + "<br>".join(numbered_list)
                + "</div>",
                unsafe_allow_html=True
            )

            # Attempt to generate a PDF flowchart using graphviz
            pdf_bytes = None
            diagram_label_name = "Flowchart (PDF)"
            if _GRAPHVIZ_AVAILABLE:
                try:
                    dot = graphviz.Digraph(format="pdf")
                    dot.attr(rankdir="TB")  # top-to-bottom

                    # Add nodes with shapes
                    for node in node_order:
                        nid = id_map[node]
                        label = f"{nid}\\n{node}"
                        low = node.lower()
                        if node == "Product Requisition":
                            dot.node(nid, label=label, shape="oval", style="filled", fillcolor="lightgrey")
                        elif ("reject" in low) or ("closed" in low):
                            dot.node(nid, label=label, shape="doublecircle", style="filled", fillcolor="lightcoral")
                        else:
                            dot.node(nid, label=label, shape="box")

                    # Build edges (aggregate labels for same pair)
                    edge_label_map = defaultdict(list)
                    for frm, lst in edges.items():
                        for to, tname in lst:
                            edge_label_map[(id_map.get(frm, frm), id_map.get(to, to))].append(tname)

                    for (f, t), labels in edge_label_map.items():
                        # join multiple labels with " | "
                        cleaned = " | ".join([l for l in labels if l])
                        if cleaned:
                            dot.edge(f, t, label=cleaned)
                        else:
                            dot.edge(f, t)

                    # Render to temporary file and read bytes
                    tmp_output = "/tmp/flowchart"
                    # cleanup=True will remove intermediate files (but graphviz python's render may leave .pdf depending)
                    rendered_path = dot.render(filename=tmp_output, cleanup=True)
                    # rendered_path should be '/tmp/flowchart.pdf'
                    with open(rendered_path, "rb") as fh:
                        pdf_bytes = fh.read()
                    # Mark diagram as generated
                    diagram_generated = True
                    diagram_bytes = pdf_bytes
                except Exception as e:
                    st.warning(f"Graphviz render failed: {e}. The Visio sheet was still created. To enable PDF exports, install Graphviz system package (e.g., apt-get install graphviz) and the python 'graphviz' package.")
                    diagram_generated = False
            else:
                st.info("Graphviz python package not found. Install system Graphviz and `pip install graphviz` to enable PDF flowchart generation.")
                diagram_generated = False

            # If we have PDF bytes, add a "Flowchart (PDF)" entry to the live list
            if diagram_generated:
                created_sheet_names.append(diagram_label_name)
                numbered_list = [f"{idx+1}. {name}" for idx, name in enumerate(created_sheet_names)]
                sheet_list_placeholder.markdown(
                    "<div style='max-height:200px; overflow-y:auto; border:1px solid #444; padding:10px; border-radius:5px;'>"
                    + "<br>".join(numbered_list)
                    + "</div>",
                    unsafe_allow_html=True
                )

            # Final progress update
            progress_bar.progress(100)

        # ----------------------------------------------------------- #

        # Save Excel to bytes and prepare downloads
        wb.save(output)
        output.seek(0)
        excel_bytes = output.getvalue()

        # Prepare filenames
        original_name = uploaded_file.name if uploaded_file is not None else "file.xlsx"
        excel_download_name = f"Processed_{original_name}"
        pdf_download_name = f"Processed_{original_name.rsplit('.',1)[0]}_flowchart.pdf"

        # Excel download button
        st.download_button(
            label="Download Modified Excel",
            data=excel_bytes,
            file_name=excel_download_name,
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )

        # PDF download button (if generated)
        if diagram_generated and diagram_bytes:
            st.download_button(
                label="Download Flowchart (PDF)",
                data=diagram_bytes,
                file_name=pdf_download_name,
                mime="application/pdf"
            )
        else:
            st.info("Flowchart PDF was not generated (Graphviz missing or rendering failed). The Transition_mapping_visio sheet was still created in the Excel file.")

        st.success("Modified Excel created successfully!")


Overwriting app.py


In [26]:
# Install packages (if not installed already)
!pip install streamlit pyngrok openpyxl --quiet

from pyngrok import ngrok
import time

# Kill previous tunnels
ngrok.kill()

# Set your ngrok token
ngrok.set_auth_token("35gXlCpbnwCAIJ9I2xiA1GXDu2u_3UoQf6QeGYhFC9MxYapgo")

# Start Streamlit in the background
get_ipython().system_raw("streamlit run app.py &")

# Wait a few seconds for Streamlit to start
time.sleep(5)

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("Streamlit app running at:", public_url)


Streamlit app running at: NgrokTunnel: "https://armless-twirly-alanna.ngrok-free.dev" -> "http://localhost:8501"
